In [1]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, dsl, Module

ws = Workspace.get(name='kubeflow_ws_1', subscription_id='74eccef0-4b8d-4f83-b5f9-fa100d155b22', resource_group='kubeflow-demo')

one2two = Module.from_yaml(ws, yaml_file='./noop/1in2out.spec.yaml')
one2one = Module.from_yaml(ws, yaml_file='./noop/1in1out.spec.yaml')
data = Dataset.get_by_name(ws, 'training_data')

In [2]:
@dsl.pipeline(
    name='2x Splits',
    description='A sample',
    default_compute_target='cpu-lowpri'
)
def cell_division():
    layer = 3
    nodes = []
    nodes.append(one2two(input1=data))
    last_layer = []
    for i in range(0, layer-1):
        print('i=', i, ' nodes len=', len(nodes))
        current_layer_nodes = []
        for j in range(0, pow(2,i)):
            print('j=', j)
            n = nodes[-j-1]
            current_layer_nodes.append(one2two(input1=n.outputs.output1))
            current_layer_nodes.append(one2two(input1=n.outputs.output2))
        nodes = nodes + current_layer_nodes
        last_layer = current_layer_nodes

    x = {}
    seq = 0
    for n in last_layer:
        seq += 1
        x['output{}'.format(seq)] = n.outputs.output1
        seq += 1
        x['output{}'.format(seq)] = n.outputs.output2
    return x

In [3]:
@dsl.pipeline(
    name='Chain',
    description='A sample',
    default_compute_target='cpu-lowpri'
)
def chain(input):
    length = 10
    last = None
    for i in range(0, length):
        j = None
        if last == None:
            j = input
        else:
            j = last.outputs.output1
        node = one2one(input1=j)
        last = node

    return {**last.outputs}

In [4]:
@dsl.pipeline(
    name='A pipeline composed with split and chain',
    description='A sample',
    default_compute_target='cpu-lowpri'
)
def waterfall():
    part1 = cell_division()
    x = {}
    for o in part1.outputs.values():
        part2 = chain(o)
        x = {**x, **part2.outputs}

    return x

In [11]:
pipeline = waterfall()

pipeline.validate()

i= 0  nodes len= 1
j= 0
i= 1  nodes len= 3
j= 0
j= 1


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

SupportDetectView()

{'result': 'validation passed', 'errors': []}

In [12]:
#pipeline.submit(experiment_name='samples')